In [2]:
library(tidyverse)
library(nycflights13)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.8
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.3.0     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()  masks stats::filter()
✖ purrr::flatten() masks jsonlite::flatten()
✖ dplyr::lag()     masks stats::lag()


# Lecture 18: More on functional programming

## Review questions
A couple of things that came up in OH and on Canvas:

### `while` vs. `for`
There were some questions in OH about `while` versus `for`. We generally use `for`. They are somewhat equivalent:
```{r}
for (i in 1:n) {
    <body>
}
```
is the same as 
```{r}
i = 1
while (i <= n) {
    <body>
    i = i + 1
}
```

### Anonymous functions
An anonymous function is, literally, a function with no name. Some of the problems on PS8 use this, e.g.:
```{r}
int01(function(x) x))
#     ^^^^^^^^^^^^^^
#   anonymous function
```
You should generally only use these for small, simple functions (max 1 line, no `{}` braces).

## Iterating over multiple sequences at once
Sometimes we want to iterate over multiple sequences. For example, suppose we had a vector `mu` of means and an equal length vector `sigma` of standard deviations. For each pair `mu[[i]],sigma[[i]]` we would like to generate a five standard normal random variable using `rnorm`.

Using `map`, we could accomplish this by

In [3]:
mu = list(5, 10, -3)
sigma = list(1, 5, 10)
seq_along(mu) %>% 
  map(~ rnorm(5, mu[[.]], sigma[[.]])) %>% 
  str()

List of 3
 $ : num [1:5] 4.96 4.98 5.94 5.82 5.59
 $ : num [1:5] 14.5949 13.9107 10.3728 0.0532 13.0991
 $ : num [1:5] -3.56 -4.56 -17.71 -7.78 1.18


Because we don't yet know how to `map` over more than one sequence at a time, we are forced to "hack it" by iterating over `seq_along(mu)`. This hides the true intent of what we set out to accomplish, which is to perform a map over *pairs* of `(mu[i], sigma[i])`.

To iterate over two sequences at once we have the `map2` command:
```{r}
map2(seq1, seq2, f, ...)
```
will call `f(seq1[[i]], seq2[[i]], ...)` for each value of `i`. Indeed, `map2` is equivalent to:
```{r}
map2 <- function(x, y, f, ...) {
  out <- vector("list", length(x))
  for (i in seq_along(x)) {
    out[[i]] <- f(x[[i]], y[[i]], ...)
  }
  out
}
```

`map2` lets us succinctly rewrite the sampling code given above:

In [196]:
map2(mu, sigma, rnorm, n = 5)

[[1]]
[1] 5.447983 5.446291 5.066780 5.075793 4.059494

[[2]]
[1]  3.518528 19.366107  7.197257  5.081700 20.487066

[[3]]
[1] -9.655409  8.922227  4.592133  4.237675 -1.451280


## Predicates
Predicates are functions that allow you to filter out elements of sequences based on a condition. The `keep(f)` command will return a new sequence consisting of each element where `f` evaluates to `TRUE`:

In [3]:
mpg %>% keep(is_integer) %>% print

# A tibble: 234 x 4
    year   cyl   cty   hwy
   <int> <int> <int> <int>
 1  1999     4    18    29
 2  1999     4    21    29
 3  2008     4    20    31
 4  2008     4    21    30
 5  1999     6    16    26
 6  1999     6    18    26
 7  2008     6    18    27
 8  1999     4    18    26
 9  1999     4    16    25
10  2008     4    20    28
# ... with 224 more rows


`discard()` does the opposite of `keep()`:

In [4]:
mpg %>% discard(is_integer) %>% print

# A tibble: 234 x 7
   manufacturer model      displ trans      drv   fl    class  
   <chr>        <chr>      <dbl> <chr>      <chr> <chr> <chr>  
 1 audi         a4           1.8 auto(l5)   f     p     compact
 2 audi         a4           1.8 manual(m5) f     p     compact
 3 audi         a4           2   manual(m6) f     p     compact
 4 audi         a4           2   auto(av)   f     p     compact
 5 audi         a4           2.8 auto(l5)   f     p     compact
 6 audi         a4           2.8 manual(m5) f     p     compact
 7 audi         a4           3.1 auto(av)   f     p     compact
 8 audi         a4 quattro   1.8 manual(m5) 4     p     compact
 9 audi         a4 quattro   1.8 auto(l5)   4     p     compact
10 audi         a4 quattro   2   manual(m6) 4     p     compact
# ... with 224 more rows


Again, these examples work because *data frames/tibble are lists*.

`detect(f)` and `detect_index(f)` return the first element (or its index) where `f` evaluates to `TRUE`:

In [54]:
rnorm(1000) %>% detect(~ . > 2)

[1] 2.329436

In [8]:
rnorm(1000) %>% detect_index(~ abs(.) > 10)

[1] 0

## The `reduce` function
The last bit of functional programming we will study is `reduce()`. The reduce function takes a “binary” function (i.e. a function with two primary inputs), and applies it repeatedly to a list until there is only a single element left. This takes a little bit of getting used to, but ends up being very powerful.

Let's take a simple example: summing up numbers. If I wanted to add the numbers 1 through 4 I could of course type `sum(1:4)` and get `10`. Suppose I did not know about sum. I could also write:

```{r}
> 1 + 2 + 3 + 4
[1] 10
```

We can rewrite this sum as 
```{r}
> (((1 + 2) + 3) + 4)
[1] 10
```
If $f(x,y) = x + y$ then we can rewrite this as 
$$f(
f(
f(1,2),
3),
4).
$$

Obviously this is a silly thing to do for computing a simple sum, but the pattern of repeatedly applying a binary function $f(x,y)$, where $y$ is the output from a previous application of the function, actually turns up a lot in programming. The `reduce` function accomplishes this for us:

In [5]:
reduce(1:4, `+`)

[1] 10

## `accumulate`
A closely related variant of `reduce` is `accumulate`. In the above example, instead of returning 

$$f(1, f(2, f(3, 4)))$$

accumulate would return a *vector* containing
all of the partially evaluated sums:

$$\texttt{accumulate(f, 1:44)} = (f(1, 2), f(f(1, 2), 3), f(f(f(1, 2), 3), 4))$$

In [8]:
1:4 %>% accumulate(`+`)

[1]  1  3  6 10

### Example
Suppose we have a list of vectors $(v_1,\dots,v_n)$ and we want to find their intersection. Mathematically this equals $$I_n:=\bigcap_{i=1}^n v_n=v_n\cap I_{n-1},$$ so we can use reduce to compute it:

In [11]:
vs = list(
  c(1, 3, 5, 6, 10),
  c(1, 2, 3, 7, 8, 10),
  c(1, 2, 3, 4, 8, 9, 10)
)
vs %>% reduce(intersect)

[1]  1  3 10

### Exercise
Let's say we want to concatenate together a bunch of strings, and because we haven't been coming to lecture we don't know about `str_c`. In fact, we only know about the following function which concatenates just two strings at a time:

In [14]:
cat2 = function(x, y, ...) {
    paste0(x, y, ...)
}

Use `cat2` and `reduce` to build a function which will concatenate whole vectors of strings together.

Let's think about what our new function will do. If we pass it the vector `c('a','b','c','d')` we should get back the string `'abcd'`. We could accomplish this using `cat2`:

In [15]:
v = c('a', 'b', 'c', 'd')
cat2(cat2(cat2(v[1], v[2]), v[3]), v[4])

[1] "abcd"

This pattern suggests how we might use `reduce`:

In [18]:
v %>% reduce(cat2)

[1] "abcd"

### Exercise
Given a vector $v$, the *cumulative product* of $v$ is a vector $p$ such that $$p_j = \prod_{i=1}^j v_i.$$
For example, the cumulative product of $(1,2,3,4)$ is $(1,2,6,24)$ (i.e. the factorials). 

Use `accumulate` to write a function `cumprod(v)` that computes the cumulative product.

In [20]:
1:10 %>% accumulate(~ .x * .y)

 [1]       1       2       6      24     120     720    5040   40320  362880
[10] 3628800

## Closures
When we create a function
```{r}
f = function(x) {
    <do somethings with x>
}
```
that function has access to:
- all of the variables defined inside the function (its arguments plus whatever other variables we create in the body of the function), *as well as*
- all of the variables defined "outside" the function.

Example:

In [29]:
y = 1
f = function(x) {
    x + y
}

Now `f` is a function that will add one to its argument:

In [30]:
f(1)

[1] 2

In fact, we can change the behavior of `f` by changing `y`. Observe:

In [34]:
y = 2
f(1)  # now f adds 2 to x

[1] 3

In [35]:
y = NA
f(1) 

[1] NA

In this example `f` is what's called a *closure*, because it *encloses* all the variables defined outside of `f`, as well as those defined inside of it.

Closures become useful when we think about creating functions inside of other functions:

In [47]:
power = function(exponent) {
  function(x) {
    x ^ exponent
  }
}
square = power(2)
cube = power(3)

What is `square` now? It's a function:

In [49]:
square(3)
cube(4)

[1] 9

[1] 64

As we said, `square` is a closure. The list of variables it encloses is called its *environment*. You can look at this list by using the command of that name:

In [50]:
square
as.list(environment(square))

function(x) {
    x ^ exponent
  }
<environment: 0x7fe8b18b6958>

$exponent
[1] 2


In [53]:
as.list(environment(cube))

$exponent
[1] 3


### Memoization
A good use for closures is something called *memoization*. Memoization lets you save the results of a function call for future use. This is very useful in combination with recursion.

To implement memoization, we define a list in the enclosing enviroment of the function we wish to memoize. Then, each time we call the function, we check to see if the value is already saved in the list. If so, we return it; otherwise, we do the computation and store it for future use.

Consider the following recursive function:

In [21]:
f = function(k) {
    if (k <= 2) {
        1
    } else {
        k^2 * f(k - 1) - k * f(k - 2)
    }
}

This function is slow to compute for large $k$:

In [22]:
library(microbenchmark)
microbenchmark(
    f(10),
    f(20)
) %>% print

Unit: microseconds
  expr      min        lq       mean    median       uq       max neval
 f(10)   40.181   42.0465   48.16137   43.6645   49.842    78.452   100
 f(20) 5014.103 5284.8700 6153.56760 5596.9620 6907.247 13576.366   100


To speed up `f` we will memoize it:

In [24]:
memo = rep(NA_real_, 100)
fmemo = function(k) {
    if (is.na(memo[[k]])) {
        ret = NA
        if (k <= 2) {
            ret = 1
        } else {
            ret = k^2 * fmemo(k - 1) - k * fmemo(k - 2)
        }
        memo[[k]] <<- ret
    }
    memo[[k]]
}

Note the use of the `<<-` operator. This is required in order to make assignments to the enclosed variable `memo`.

In [25]:
library(microbenchmark)
microbenchmark(
    f(10),
    f(20),
    fmemo(10),
    fmemo(20)
) %>% print

Unit: nanoseconds
      expr     min        lq       mean  median        uq     max neval
     f(10)   40626   41916.0   45247.92   42941   46218.5   68022   100
     f(20) 5050269 5237409.0 5659128.15 5334692 5606398.5 9841770   100
 fmemo(10)     542     703.5   85840.19     841    2830.5 8390247   100
 fmemo(20)     566     699.5    2150.37     905    3075.5   20581   100


## Application: analyzing cancer expression data
Why do we care about functional programming? Here is a real-life example.

In [9]:
# ### training data
# filename <- paste("http://pubs.broadinstitute.org/mpr/projects/",
# "Global_Cancer_Map/GCM_Training.res", sep="")
# dat0 <- read.delim(filename, sep="\t", header=FALSE, skip=3, quote="")
# tmp <- dat0[,1:290]
# tmp <- tmp[, -seq(4, 290, by=2)]
# tmp <- tmp[, -(1:2)]
# train <- t(tmp)
# filename <- paste("http://pubs.broadinstitute.org/mpr/projects/",
# "Global_Cancer_Map/GCM_Training.cls", sep="")
# train.classes <- read.table(filename, skip=2)+1
# train.classes <- unlist(train.classes)
# ### test data
# filename <- paste("http://pubs.broadinstitute.org/mpr/projects/",
# "Global_Cancer_Map/GCM_Test.res", sep="")
# dat0 <- read.delim(filename, sep="\t", header=FALSE, skip=3, quote="")
# tmp <- dat0[,1:110]
# tmp <- tmp[, -seq(4, 110, by=2)]
# tmp <- tmp[, -(1:2)]
# test <- t(tmp)[1:46,]
# filename <- paste("http://pubs.broadinstitute.org/mpr/projects/",
# "Global_Cancer_Map/GCM_Test.cls", sep="")
# test.classes <- read.table(filename, skip=2)+1
# test.classes <- test.classes[test.classes!=15]
# test.classes <- unlist(test.classes)
# tr <- as_tibble(train) %>% mutate(class = train.classes, split="train")
# ts <- as_tibble(test) %>% mutate(class = test.classes, split="test")
# gcm14 <- bind_rows(tr, ts)